# **EXPLORER FOR SEC FILINGS**
<hr>

## Inter IIT Tech Meet 10.0 (2022)

![image](https://www.sec.gov/edgar/search/images/edgar-logo-2x.png)
![image](https://interiit-tech.org/static/media/logo_1.f4d40e83.png)

In this Notebook, we shall be looking into utilizing the [EDGAR](https://www.sec.gov/edgar/searchedgar/) API to explore the SEC filings of a company. We shall be using the python library python-edgar to access the API. Be careful, the API is limited to 10 requests per second or smth, idk. If a black SUV shows up out in the open, it's probably because you're doing something wrong.

## Objectives
- Scrape Data from the company's History since inception
- Use 10-Q 10-K and 8-K filings to get the company's financial statements
- Use Financial Statements to get the company's balance sheet, income statement, cash flow statement, and ratios
- Use the data to get the company's current assets, liabilities, and equity
- Generate SaaS Metrics
- Generate a Financial Statement Analysis
- Use the metrics with Deep Learning Systems to give Insightful Results

Graciaz<br>
Kaushik Dey


### Setup and Importing Modules
We will need
- Edgar for the API
- Pandas for data manipulation
- Matplotlib for plotting
- OS for file system paths

In [1]:
import edgar
import requests
import pandas as pd
import matplotlib.pyplot as plt
import pandasgui as pdgui
import os
import numpy as np
import json
import time

### Importing SaaS Companies
#### (Optional if you have the Output Files Already)

- Get the name of the required companies from the databbase
- Find CIK Number for each company
- Put it in a table

Current implementation takes anywhere between 12-20 minutes to run, so feel free to optimize the algorithm if you have time

In [ ]:
cwd = os.getcwd()
path = os.path.join(cwd, 'data','edgar-data')

#Find Company Names
company_names = pd.read_csv("data/company_list.csv")
company_names["CIK"] = "";
# Define cik_list
#Its a percent-dollar seperated values, sorry for that
all_cik = pd.read_csv("data/all_cik.pdsv", sep="^", lineterminator='\n')

found = False #To highlight if a company is found

ciklength = len(all_cik.index) # Number of CIK Entries

itr = 0
previtr = 5

for i in company_names.index:
    found = False
    itr = 0
    while itr < ciklength:
        try:
            if company_names["Company"][i] == all_cik["NAME"][itr]:
                print("found it! (", i+1 , ") " , all_cik["NAME"][itr], "at line", itr+2 )
                company_names["CIK"][i] = all_cik["CIK"][itr]
                found = True
                previtr = itr
                break
            itr += 1
        except:
            print("Error in company name: " , company_names["Company"][i] , " and " , all_cik["NAME"][itr] , " at cik file line " , itr+2)
    if not found:
        print("Not found: " , company_names["Company"][i])
        # print("Starting Again From: " , itr+2, " ", all_cik["NAME"][previtr])

company_names.to_csv("data/output.csv", index=False)
        

### Importing SaaS Companies

#### (if you don't have the Output Files Already go to previous step)

- Import data
- Fill NaN with zeroes
- convert to int64
- Save to CSV(optional)

In [2]:
company_names = pd.read_csv("data/output.csv")
company_names["CIK"] = company_names["CIK"].fillna(0).apply(np.int64)
for i in company_names.index:
    company_names["CIK"][i] = str(company_names["CIK"][i]).zfill(10)
# company_names.to_csv("data/output.csv", index=False)
company_names

C:\Users\Kaushik Dey\AppData\Local\Temp\ipykernel_12844\571149940.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  company_names["CIK"][i] = str(company_names["CIK"][i]).zfill(10)


,Company,CIK
0,2U INC,0001459417
1,30DC INC,0001118974
2,3D PIONEER SYSTEMS INC,0001556753
3,8X8 INC,0001023731
4,A10 NETWORKS INC,0001580808
...,...,...
287,ZOOM VIDEO COMUNICATIONS INC,0001585521
288,ZOOMAWAY TRAVEL INC,0001541054
289,ZOOMINFO TECHNOLOGIES,0001794515
290,ZSCALER INC,0001713683


### Downloading Filings(Optional)

#### Using `python-edgar` to access the API
- Completely Selective Step


In [ ]:
edgar.download_index(path,1996,"Indian Institute of Technology, Bhubaneswar 20CS01043@iitbbs.ac.in");
print("Downloaded Files saved at: ", path)

### Add JSON Metadata to the Companies
- Update and add `HISTORY` Column to the table
- Add the JSON Data files to the table
- Add the JSON Data to the table

In [9]:
company_names["HISTORY"] = "";

head = {
    "User-Agent": "Alpha-Explorer/1.0"
}

for i in company_names.index:
    try:
        if company_names["CIK"][i] != 0:
            url = "https://data.sec.gov/submissions/CIK"+company_names["CIK"][i]+".json" #Link to JSON file
            response= requests.get(url, headers=head)
            time.sleep(0.1)
            company_names["HISTORY"][i] = response.json()
    except:
        print("Error in company name: " , company_names["Company"][i])


Error in company name:  FLEXSHARES IBOXX 3-YR TAR FD
Error in company name:  FLEXSHARES IBOXX 5-YR TAR FD
Error in company name:  ISHARES IBOXX HIGH YLD CP BD
Error in company name:  ISHARES IBOXX INVST GR CP BD
Error in company name:  MEDALLIANCE INC


### STORE IN A FILE
Some JSON Reformats

In [14]:
for i in company_names.index:
    company_names["HISTORY"][i] = json.dumps(company_names["HISTORY"][i])
company_names.to_csv("data/company_summary.csv", index=False)
